In [ ]:
#Importing all the required Packages
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, svm
from sklearn import metrics
from scipy.stats import f_oneway
from matplotlib.pyplot import plot, show
from sklearn.preprocessing import OneHotEncoder
from wordcloud import WordCloud

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#Reading every sheet
df19_20= pd.read_excel("/content/drive/My Drive/Colab Notebooks/UMD Alumni Association Dataset.xlsx", sheet_name="2019-20")
df18_19= pd.read_excel("/content/drive/My Drive/Colab Notebooks/UMD Alumni Association Dataset.xlsx", sheet_name="2018-19")
df17_18= pd.read_excel("/content/drive/My Drive/Colab Notebooks/UMD Alumni Association Dataset.xlsx", sheet_name="2017-18")
df16_17= pd.read_excel("/content/drive/My Drive/Colab Notebooks/UMD Alumni Association Dataset.xlsx", sheet_name="2016-17")
df15_16= pd.read_excel("/content/drive/My Drive/Colab Notebooks/UMD Alumni Association Dataset.xlsx", sheet_name="2015-16")
df14_15= pd.read_excel("/content/drive/My Drive/Colab Notebooks/UMD Alumni Association Dataset.xlsx", sheet_name="2014-15")
df13_14= pd.read_excel("/content/drive/My Drive/Colab Notebooks/UMD Alumni Association Dataset.xlsx", sheet_name="2013-14")

# Data Manipulation / Cleansing / Preprocessing

##Merging into 1 df

In [ ]:
#Concatenating sheet into single dataframe
#https://stackoverflow.com/questions/25486438/how-to-dynamically-refer-to-dataframes-in-a-for-loop-in-python
df=pd.concat([df19_20,df18_19,df17_18,df16_17,df15_16,df14_15,df13_14],ignore_index=True)
df.shape

##Renaming Attributes

In [ ]:
#https://www.listendata.com/2020/09/How-to-rename-columns-in-Pandas.html
df.columns = df.columns.str.replace(' ' , '_')
df

##Adding quarter atrribute

In [ ]:
#Create new column "quarter" which extracts Event_Date and returns quarter
#https://stackoverflow.com/questions/1406131/is-there-a-python-function-to-determine-which-quarter-of-the-year-a-date-is-in
df['quarter'] = df['Event_Date'].dt.quarter
df[["quarter"]]

##Extracting Month and Year from Event Date

In [ ]:
#referenced pandas documentation for datetime manipulation
#https://stackoverflow.com/questions/25146121/extracting-just-month-and-year-separately-from-pandas-datetime-column
df['year'] = pd.DatetimeIndex(df['Event_Date']).year
df['month'] = pd.DatetimeIndex(df['Event_Date']).month

##Adding age_bin attribute

In [ ]:
#checking bounds for average_age to decide bins
df["Average_Age"].describe()

In [ ]:
#referenced pandas documentation to use pd.cut to create bins
#https://www.geeksforgeeks.org/pandas-cut-method-in-python/
df['age_bins'] = pd.cut(x=df['Average_Age'], bins=[22, 29, 39, 49, 59, 69, 79])
df['age_bins'].value_counts()

##Check for null values

In [ ]:
#Check for null
df.isnull().sum()
df.head(5)

#Descriptive Stastics

##Check Data Types in df

In [ ]:
#check data types
df.dtypes

##Summary Statistics

In [ ]:
#Summary Statistics
df.describe()

##Histogram for data distribution

In [ ]:
#Check distribution of numerical variables
plt.figure(figsize=(17,5))
plt.suptitle("Distribution of Numerical Data", size=20)

plt.subplot(231)
plt.hist(df["Average_Age"])
plt.title("Average Age Dsitribution")

plt.subplot(232)
plt.hist(df["First_Time_Attendees"])
plt.title("First_Time_Attendees")

plt.subplot(233)
plt.hist(df["Major_Prospects"])
plt.title("Major_Prospects")

plt.subplot(234)
plt.hist(df["Participated"])
plt.title("Particiaption Distribution")

plt.subplot(235)
plt.hist(df["Percentage_First_Time_Attendees"])
plt.title("Percentage_First_Time_Attendees")

plt.subplot(236)
plt.hist(df["Percentage_Major_Prospect"])
plt.title("Percentage_Major_Prospect")

plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

* Average age is the only variable with a normal distribution. 
* All others are right-skewed and outliers within them make their shape look so extreme. 
For further analysis, this data can be normalized by taking log of the variables or exploring removing those outliers.


##Correlation and Heatmap

In [ ]:
#correlation pairwise plot
plt.figure(figsize=(11,19))
pairwise=sns.pairplot(df,palette="husl",vars=['Participated', 'Average_Age','First_Time_Attendees','Major_Prospects','Percentage_Major_Prospect'],kind="reg",height=4, aspect=1)
pairwise.fig.suptitle("Correlation Pairwise Plot",y=1, fontsize=30)
plt.show()

In [ ]:
#Correlation Heatmap
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12)

#Participation Metrics

##By Event

In [ ]:
#Treating inconsistency in Activity description
#https://www.geeksforgeeks.org/python-string-strip-2/
df["Activity_Description"] = df["Activity_Description"].str.strip()
df["Activity_Description"] = df["Activity_Description"].str.replace("- ","-")

In [ ]:
#Overall particiaption by event
df.groupby(["Event_Name","Activity_Description"])["Participated"].sum().sort_values(ascending=False).head(5)

In [ ]:
#Grouping by Event Name
df_enad=df[['Event_Name','Activity_Description','Participated','First_Time_Attendees','Major_Prospects']]
df_enad.groupby(["Event_Name","Activity_Description"]).agg({'Participated':['sum'],'First_Time_Attendees':['sum'],'Major_Prospects':['sum']})
df_enad.columns=["Event_Name","Activity_Description",'Participated_monthly','First_Time_attendees_monthly','Major_Prospects']
df_enad=df_enad.reset_index()

In [ ]:
#Extracting activity from Activity description
df['Activity_extracted'] = (df['Activity_Description'].str.split('-').str[-1])
df['Activity_extracted']= df['Activity_Description'].str.strip()
df['Activity_extracted'] 

In [ ]:
# Grouping by activity of the event
df_Activity_Participated=(df.groupby(['Activity_extracted'],as_index=False)['Participated'].sum()).sort_values('Participated')
df_Activity_Participated
df_Activity_first_time=(df.groupby(['Activity_extracted'],as_index=False)['First_Time_Attendees'].sum()).sort_values('First_Time_Attendees')
df_Activity_first_time
df_Activity_Major_prospects=(df.groupby(['Activity_extracted'],as_index=False)['Major_Prospects'].sum()).sort_values('Major_Prospects')
df_Activity_Major_prospects

In [ ]:
# Reference https://amueller.github.io/word_cloud/
dict1=dict(zip(df_Activity_Participated.Activity_extracted, df_Activity_Participated.Participated))
word_could_dict = dict1
wordcloud = WordCloud(width = 1000, height = 500).generate_from_frequencies(word_could_dict)
fig= plt.figure(figsize=(15,8))
ax=plt.subplot(1,1,1,xticks=[ ],yticks=[ ])
plt.imshow(wordcloud)
plt.title('Wordcloud of event Name according to number of participants')
plt.show()

In [ ]:
#how often does event happen
df.groupby(["Event_Name"])["Activity_Description"].value_counts().sort_values(ascending=False).head(10)

* DC Terps Game Watch and Baltimore Terps Game Watch  are the most frequent events



##By Location

In [ ]:
#Participation by Location- drop location description
df.groupby(["Location_Description","Location_Code"])["Participated"].sum().sort_values(ascending=False).head(5)

In [ ]:
#look for inconsistency
Location_Description=df["Location_Description"]
sorted(Location_Description.unique())

In [ ]:
#Extracting region from location description
df['region_Extracted'] = (df['Location_Description'].str.split('-').str[0])
df['region_Extracted']=df['region_Extracted'].str.strip()
df['region_Extracted']=df['region_Extracted'].str.split(' ').str[-1]


In [ ]:
#Extracting location from Location description
df['Location_Extracted'] = (df['Location_Description'].str.split('-').str[-1])
df['Location_Extracted']= df['Location_Extracted'].str.strip()
df['Location_Extracted'] 

In [ ]:
#Grouping by Region
df_location_region_participated=(df.groupby(['region_Extracted'],as_index=False)['Participated'].sum()).sort_values('Participated')
df_location_region_firsttime=df.groupby(['region_Extracted'],as_index=False)['First_Time_Attendees'].sum().sort_values('First_Time_Attendees')
df_location_region_majorprospects=df.groupby(['region_Extracted'],as_index=False)['Major_Prospects'].sum().sort_values('Major_Prospects')
df_location_region_firsttime_and_participated=df_location_region_participated .merge(df_location_region_firsttime)
df_location_region_all=df_location_region_firsttime_and_participated.merge(df_location_region_majorprospects)
#Treating inconsistency
#https://www.journaldev.com/37898/python-numpy-where
df['region_Extracted'] = np.where((df['region_Extracted'] == 'Norhteast'),'Northeast',df['region_Extracted'])
df['region_Extracted'] = np.where((df['region_Extracted'] == 'DC'),'DMV',df['region_Extracted'])


In [ ]:
# Extraction of Location from location description
# Grouping by Participants, first time and Major prospects
df_location_participated=df.groupby(['Location_Extracted'],as_index=False)['Participated'].sum()
df_location_firsttime=df.groupby(['Location_Extracted'],as_index=False)['First_Time_Attendees'].sum()
df_location_majorprospects=df.groupby(['Location_Extracted'],as_index=False)['Major_Prospects'].sum()
df_location_firsttime_and_participated=df_location_participated .merge(df_location_firsttime)
df_location_all=df_location_participated.merge(df_location_majorprospects)
#Treating Indiscrepancy
df['Location_Extracted'] = np.where((df['Location_Extracted'] == 'Dallas/Forth Worth'),'Dallas/Fort Worth',df['Location_Extracted'])
df['Location_Extracted'] = np.where((df['Location_Extracted'] == 'Prince Georges County'),"Prince George's County",df['Location_Extracted'])
df['Location_Extracted'] = np.where((df['Location_Extracted'] == 'DC'),"Washington DC",df['Location_Extracted'])
df['Location_Extracted'] = np.where((df['Location_Extracted'] == 'Prince Georges County'),"Prince George's County",df['Location_Extracted'])
df['Location_Extracted'] = np.where((df['Location_Extracted'] == 'Southern MD'),"Southern Maryland",df['Location_Extracted'])
df['Location_Extracted'] = np.where((df['Location_Extracted'] == 'Northen VA'),"Northern VA",df['Location_Extracted'])
df['Location_Extracted'] = np.where((df['Location_Extracted'] == 'Northern VA'),"Northern Virginia",df['Location_Extracted'])
df['Location_Extracted'] = np.where((df['Location_Extracted'] == 'San Francsico'),"San Francisco",df['Location_Extracted'])
df['Location_Extracted'] = np.where((df['Location_Extracted'] == 'Washinton DC'),"Washington DC",df['Location_Extracted'])

In [ ]:
#Plotting Number of Participants with Region of event
plt.figure(figsize=(15,7))
plt.barh(df_location_region_participated['region_Extracted'],df_location_region_participated['Participated'],color=['red'])
plt.ylabel("Region of event")
plt.xlabel("Number of Participants")
plt.title("Number of participants verssus Region of event")
plt.show()

In [ ]:
#Count of the regions
df['region_Extracted'].value_counts()

In [ ]:
#Number of participants vs Location of event
plt.figure(figsize=(14,7))
plt.bar(df_location_firsttime_and_participated['Location_Extracted'],df_location_firsttime_and_participated['Participated'],color=['red'])
plt.xlabel("Location of Event")
plt.ylabel("Number of Participants")
plt.title("Number of participants vs Location of event")
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Count of Locations
df['Location_Extracted'].value_counts()

##By Age

In [ ]:
#Grouping by Age Bins
df_age_participated=df.groupby(['age_bins'],as_index=False)['Participated'].sum()
df_age_firsttime=df.groupby(['age_bins'],as_index=False)['First_Time_Attendees'].sum()
df_age_majorprospects=df.groupby(['age_bins'],as_index=False)['Major_Prospects'].sum()
df_age_all=df_age_participated.merge(df_age_firsttime)
df_age_all['Percentage_first_time']=df_age_all['First_Time_Attendees']/df_age_all['Participated']

In [ ]:
#plotting Number of participants with Age 
cmap = plt.get_cmap("tab20c")
my_circle = plt.Circle( (0,0), 0.3, color='white')
outer_colors = cmap(np.arange(3)*4)
plt.pie(df_age_all['Participated'], labels=df_age_all['age_bins'],colors=outer_colors,autopct='%.2f')
plt.title("Number of participants vs age group")
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

## By Group Description

 * The Group description consists information about the Purpose of the Event and also info of the Intended audience of the Event

In [ ]:
#Extracting Purpose from group code
df['purpose_Extracted'] = (df['Group_Description'].str.split('-').str[0])
df['purpose_Extracted']=df['purpose_Extracted'].str.strip()
df['purpose_Extracted']=df['purpose_Extracted'].str.split(' ').str[-1]
df['purpose_Extracted'].value_counts()
#Treating inconsistency in data
df['purpose_Extracted'] = np.where((df['purpose_Extracted'] == 'Prodev'),'ProDev',df['purpose_Extracted'])
df['purpose_Extracted'] = np.where((df['purpose_Extracted'] == 'Stewarship'),'Stewardship',df['purpose_Extracted'])
df['purpose_Extracted'] = np.where((df['purpose_Extracted'] == 'Affiniy'),'Affinity',df['purpose_Extracted'])
df['purpose_Extracted'].value_counts()
#Grouping by Purpose of the Event
df_group_purpose_participated=df.groupby(['purpose_Extracted'],as_index=False)['Participated'].sum()
df_group_purpose_firsttime=df.groupby(['purpose_Extracted'],as_index=False)['First_Time_Attendees'].sum()
df_group_purpose_majorprospects=df.groupby(['purpose_Extracted'],as_index=False)['Major_Prospects'].sum()
df_purpose_firsttime_and_participated=df_group_purpose_firsttime .merge(df_group_purpose_participated)
df_purpose_all=df_purpose_firsttime_and_participated.merge(df_group_purpose_majorprospects)
df_purpose_firsttime_and_participated['Percentage_first_time']=df_purpose_firsttime_and_participated['First_Time_Attendees']/df_purpose_firsttime_and_participated['Participated']


In [ ]:
#Extracting audience from group code- TARGET AUDIENCE
df['Extracted_Group'] = (df['Group_Description'].str.split('-').str[-1])
df['Extracted_Group'].value_counts().head()
df['Extracted_Group']=df['Extracted_Group'].str.strip()
df['Extracted_Group'].value_counts()
#Removing redundant Intended audience names -data cleaning
df['Extracted_Group'] = np.where((df['Extracted_Group'] == 'Student'),'Students',df['Extracted_Group'])
df['Extracted_Group'] = np.where((df['Extracted_Group'] == 'Emeriti'),'Emiriti',df['Extracted_Group'])
df['Extracted_Group'] = np.where((df['Extracted_Group'] == 'Genera'),'General',df['Extracted_Group'])
df['Extracted_Group'] = np.where((df['Extracted_Group'] == 'StuAlum'),'Students & Alumni',df['Extracted_Group'])
df['Extracted_Group'] = np.where((df['Extracted_Group'] == 'Students'),'Students & Alumni',df['Extracted_Group'])
df['Extracted_Group'] = np.where((df['Extracted_Group'] == 'Alumnae'),'Students & Alumni',df['Extracted_Group'])
df['Extracted_Group'].value_counts()

#Extracting intended audience from Group description
df_group_ep=df.groupby(['Extracted_Group'],as_index=False)['Participated'].sum()
df_group_ef=df.groupby(['Extracted_Group'],as_index=False)['First_Time_Attendees'].sum()
df_group_major_prospects=df.groupby(['Extracted_Group'],as_index=False)['Major_Prospects'].sum()
df_audience_firsttime_and_participated=df_group_ef .merge(df_group_ep)
df_audience_all=df_audience_firsttime_and_participated.merge(df_group_major_prospects)
df_audience_firsttime_and_participated['Percentage_first_time']=df_audience_firsttime_and_participated['First_Time_Attendees']/df_audience_firsttime_and_participated['Participated']

In [ ]:
#Plotting Number of participants with Intended audience
plt.figure(figsize=(15,7))
plt.bar(df_group_ep ['Extracted_Group'],df_group_ep ['Participated'],color=['red'])
plt.xlabel("Event's intended audience")
plt.ylabel("Number of participants ")
plt.title("Number of participants vs Intended audience")
plt.xticks(rotation=45,ha='right')
plt.show()




In [ ]:
#Plotting number of Participants with Purpose of the event
plt.figure(figsize=(15,7))
plt.style.use('default')
plt.bar(df_group_purpose_participated['purpose_Extracted'],df_group_purpose_participated['Participated'],color=['red'])
plt.xlabel("Purpose of event")
plt.ylabel("Number of Participants")
plt.title("Number of Participants verssus Purpose of event")
plt.xticks(rotation=45)
plt.show()

## By Month of the Event Date

In [ ]:
df1=df[['Percentage_First_Time_Attendees','Participated','First_Time_Attendees','Major_Prospects','year','month','Event_Date']]

In [ ]:
#Grouping the data by year and month
df_ym=df1.groupby(['year','month']).agg({'Participated':['sum'],'First_Time_Attendees':['sum'],'Major_Prospects':['sum']})
df_ym.columns=['Participated_monthly','First_Time_attendees_monthly','Major_Prospects']
df_ym=df_ym.reset_index()

In [ ]:
#Grouping monthly to calculate number of participants, First time attendees and number of Major prospects every month.
df_ym_participated=df1.groupby(['month'],as_index=False)['Participated'].sum()
df_ym_firsttime=df1.groupby(['month'],as_index=False)['First_Time_Attendees'].sum()
df_ym_majorprospects=df1.groupby(['month'],as_index=False)['Major_Prospects'].sum()

#Merging dataframes to calculate the percentage of first tie attendees every month
df_ym_all=df_ym_participated.merge(df_ym_firsttime)
df_ym_all['Percentage_first_time']=df_ym_all['First_Time_Attendees']/df_ym_all['Participated']

#Calculating percentage of first time attendees
df_ym_all['Percentage']=df_ym_all['First_Time_Attendees']/df_ym_all['Participated']


In [ ]:
#Plotting number of Participants each month 
#https://www.geeksforgeeks.org/seaborn-lineplot-method-in-python/
sns.set(rc = {'figure.figsize':(12,7)})
plt.style.use('default')
plot=sns.lineplot(df_ym_participated['month'],df_ym_participated['Participated'],color='r',linewidth=2.5)
plot.set(xlabel='Month of the year',ylabel='Number of Participants ',title='Number of Participants verssus month')
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12],['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.grid(True)
plt.show()

## By Day of the Event

In [ ]:
#Extracting day of the event Date
#https://www.geeksforgeeks.org/python-pandas-to_datetime/
df1['Day_of_Week'] = pd.to_datetime(df1['Event_Date'], format='%m/%d/%y').dt.day_name()
df1.head()

In [ ]:
#Number of Participants every Day during all the years
df_day=df1
df_day_participated=df1.groupby(['Day_of_Week'],as_index=False)['Participated'].sum()
df_day_firsttime=df1.groupby(['Day_of_Week'],as_index=False)['First_Time_Attendees'].sum()
df_day_majorprospects=df1.groupby(['Day_of_Week'],as_index=False)['Major_Prospects'].sum()
df_day_all=df_day_participated.merge(df_day_firsttime)
df_day_all['Percentage_first_time']=df_day_all['First_Time_Attendees']/df_day_all['Participated']
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
sns.set(rc = {'figure.figsize':(12,7)})
plt.style.use('default')
plot=sns.barplot(df_day_participated['Day_of_Week'],df_day_participated['Participated'],order = day_order,ci=None,color='red')
plot.set(xlabel='Day',ylabel='Participated',title='Number of attendees vs Day of the week')
plt.xticks(rotation=45,ha='right')
plt.show()



#First Time Attendence Metrics

##By Event

In [ ]:
#overall
df.groupby(["Event_Name"])["First_Time_Attendees"].sum().sort_values(ascending=False).head(15)

In [ ]:
#Word cloud of Event according to number of first time attendees
dict1=dict(zip(df_Activity_first_time.Activity_extracted, df_Activity_first_time.First_Time_Attendees))
word_could_dict = dict1
wordcloud = WordCloud(width = 1000, height = 500).generate_from_frequencies(word_could_dict)
fig= plt.figure(figsize=(15,8))
ax=plt.subplot(1,1,1,xticks=[ ],yticks=[ ])
plt.imshow(wordcloud)
plt.title('Word cloud of Event according to number of first time attendees' )
plt.show()


##By location

In [ ]:
#Plotting Number of first time attendees with Region of event
plt.figure(figsize=(15,7))
plt.barh(df_location_region_firsttime['region_Extracted'],df_location_region_firsttime['First_Time_Attendees'],color=['red'])
plt.ylabel("Region of event")
plt.xlabel("Number of first time attendees")
plt.title("Number of first time attendees verssus Region of event")
plt.show()

In [ ]:
#Number of first time attendees  vs Location of event
plt.figure(figsize=(14,7))
plt.bar(df_location_firsttime_and_participated['Location_Extracted'],df_location_firsttime_and_participated['First_Time_Attendees'],color=['red'])
plt.xlabel("Location of Event")
plt.ylabel("Number of first time attendees")
plt.title("Number of first time attendees  vs Location of event")
plt.xticks(rotation=90)
plt.show()

##By Age

In [ ]:
#Number of first time attendees 
#https://medium.com/@kvnamipara/a-better-visualisation-of-pie-charts-by-matplotlib-935b7667d77f
cmap = plt.get_cmap("tab20c")
my_circle = plt.Circle( (0,0), 0.3, color='white')
outer_colors = cmap(np.arange(3)*4)
from palettable.colorbrewer.qualitative import Pastel1_7
plt.pie(df_age_all['First_Time_Attendees'], labels=df_age_all['age_bins'],colors=outer_colors,autopct='%.2f')
plt.title("Number of first time attendees vs age group")
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

## By Group Description

In [ ]:
#Plotting Number of first time attendees with Intended audience
plt.figure(figsize=(15,7))
plt.bar(df_group_ef ['Extracted_Group'],df_group_ef ['First_Time_Attendees'], color=['red'])
plt.xlabel("Event's intended audience")
plt.ylabel("Number of First time atendees")
plt.title("Number of first time attendees vs Intended audience")
plt.xticks(rotation=45,ha='right')
plt.show()

In [ ]:
#Plotting Purpose of the event with Number of first time attendees
plt.figure(figsize=(15,7))
plt.bar(df_group_purpose_firsttime['purpose_Extracted'],df_group_purpose_firsttime['First_Time_Attendees'],color='red')
plt.xlabel("Purpose of event")
plt.ylabel("Number of First Time Attendees")
plt.title("Number of First Time Attendees verssus Purpose of event")
plt.xticks(rotation=45)
plt.show()

## By Month of the Event Date

In [ ]:
#Plotting Total Number of First time attendees each month
sns.set(rc = {'figure.figsize':(12,7)})
plt.style.use('default')
plot=sns.lineplot(df_ym_firsttime['month'],df_ym_firsttime['First_Time_Attendees'],color='r',linewidth=2.5)
plot.set(xlabel='Month of the year',ylabel='Number of first time attendees',title='Number of First time attendees verssus month')
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12],['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.grid(True)
plt.show()



## By Day of the event

In [ ]:
#Number of First time attendees Every Day during all the years
sns.set(rc = {'figure.figsize':(12,7)})
plt.style.use('default')
plot=sns.barplot(df_day_firsttime['Day_of_Week'],df_day_firsttime['First_Time_Attendees'],order = day_order,ci=None,color='red')
plot.set(xlabel='Day',ylabel='Number of First time atendees',title='Number of first time attendees vs Day of the week')
plt.xticks(rotation=45,ha='right')
plt.show()



#Major Prospects Metrics

##By Event

In [ ]:
#overall
df.groupby(["Event_Name"])["Major_Prospects"].sum().sort_values(ascending=False).head(5)

In [ ]:
#Word cloud of Event according to number of major prospects
#https://stackoverflow.com/questions/43145199/create-wordcloud-from-dictionary-values
dict1=dict(zip(df_Activity_Major_prospects.Activity_extracted, df_Activity_Major_prospects.Major_Prospects))
word_could_dict = dict1
wordcloud = WordCloud(width = 1000, height = 500).generate_from_frequencies(word_could_dict)
fig= plt.figure(figsize=(15,8))
ax=plt.subplot(1,1,1,xticks=[ ],yticks=[ ])
plt.imshow(wordcloud)
plt.title('Word cloud of Event according to number of major prospects' )
plt.show()


##By location

In [ ]:
#Plotting Number of major prospects with Region of event
plt.figure(figsize=(15,7))
plt.barh(df_location_region_all['region_Extracted'],df_location_region_all['Major_Prospects'],color=['red'])
plt.ylabel("Region of event")
plt.xlabel("Number of Major Prospects")
plt.title("Number of Number of Major Prospects verssus Region of event")
plt.show()


In [ ]:
#Number of Major prospects vs Location of event
plt.figure(figsize=(15,7))
plt.bar(df_location_all['Location_Extracted'],df_location_all['Major_Prospects'],color=['red'])
plt.xlabel("Location of Event")
plt.ylabel("Number of Major Prospects")
plt.title("Number of major prospects  vs Location of event")
plt.xticks(rotation=90)
plt.show()

##By Age

In [ ]:
#Plotting Age with Number of Major prospects

cmap = plt.get_cmap("tab20c")
my_circle = plt.Circle( (0,0), 0.3, color='white')
outer_colors = cmap(np.arange(3)*4)
plt.pie(df_age_majorprospects['Major_Prospects'], labels=df_age_majorprospects['age_bins'],colors=outer_colors,autopct='%.2f')
plt.title("Number of major prospects vs age group")
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()



## By Group Description

In [ ]:
#Major prospects according to Purpose of the Event
plt.figure(figsize=(15,7))
plt.bar(df_purpose_all['purpose_Extracted'],df_purpose_all['Major_Prospects'],color=['red'])
plt.xticks(rotation=45)
plt.xlabel("Purpose of event")
plt.ylabel("Number of Major prospects")
plt.title("Number of major prospects vs Purpose of event")
plt.show()

In [ ]:
#plotting Number of Major prospects with Intended audience
plt.figure(figsize=(15,7))
plt.bar(df_audience_all['Extracted_Group'],df_audience_all['Major_Prospects'],color=['red'])
plt.xlabel("Event intended audience")
plt.ylabel("Number of Major Prospects")
plt.title("Number of major prospects verssus Intended audience")

plt.xticks(rotation=45)
plt.show()

## By Month of the Event

In [ ]:
#Major Prospects according to Month
sns.set(rc = {'figure.figsize':(12,7)})
plt.style.use('default')
plot=sns.lineplot(df_ym_majorprospects['month'],df_ym_majorprospects['Major_Prospects'],color='r',linewidth=2.5)
plot.set(xlabel='Month of the year',ylabel='Number of Major Prospects',title='Number of Major prospects verssus Month')
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12],['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.grid(True)
plt.show()



## By Day of the Event

In [ ]:
#Plotting Major Prospects with Day of week

sns.set(rc = {'figure.figsize':(12,7)})
plt.style.use('default')
plot=sns.barplot(df_day_majorprospects['Day_of_Week'],df_day_majorprospects['Major_Prospects'],order = day_order,ci=None,color='red')
plot.set(xlabel='Day',ylabel='Number of Major Prospects',title='Number of Major Prospects verssus Day of the week')
plt.xticks(rotation=45,ha='right')
plt.show()

#ANOVA TEST

To Identify variables that are correlated to our desired outcomes of higher event attendance of first time attendees and major gift prospect attendees.

In [ ]:
#choose "useful" variables 
df_test = df[['Event_Name', 'Activity_Code', 'Location_Code',
       'Group_Code', 'Event_Date', 'Participated', 'Average_Age', 'First_Time_Attendees',
       'Percentage_First_Time_Attendees', 'Major_Prospects',
       'Percentage_Major_Prospect', 'year', 'age_bins','quarter']]

In [ ]:
target_variable = 'Major_Prospects'

for i in ['Activity_Code', 'Location_Code', 'Group_Code', 'Average_Age', 'First_Time_Attendees','quarter']:
    print('Feature Name: %s'%i)
# A one way ANOVA is used to compare two means from two independent (unrelated) groups using the F-distribution. 
# The null hypothesis for the test is that the two means are equal. Therefore, a significant result means that the two means are unequal.
# Assumption(H0) is that Percentage_Major_Prospect and Prospect are NOT correlated
 
    CategoryGroupLists=df_test.groupby(i)[target_variable].apply(list)
# Performing the ANOVA test
# We accept the Assumption(H0) only when P-Value &gt; 0.05
    AnovaResults = f_oneway(*CategoryGroupLists)
    print('P-Value for Anova is: ', AnovaResults[1])

In [ ]:
target_variable = 'First_Time_Attendees'

for i in ['Activity_Code', 'Location_Code', 'Group_Code', 'Average_Age', 'Major_Prospects','quarter']:
    print('Feature Name: %s'%i)
# A one way ANOVA is used to compare two means from two independent (unrelated) groups using the F-distribution. 
# The null hypothesis for the test is that the two means are equal. Therefore, a significant result means that the two means are unequal.
# Assumption(H0) is that Percentage_Major_Prospect and Prospect are NOT correlated
 
# Finds out the Prices data for each FuelType as a list
    CategoryGroupLists=df_test.groupby(i)[target_variable].apply(list)
# Performing the ANOVA test
# We accept the Assumption(H0) only when P-Value &gt; 0.05
    AnovaResults = f_oneway(*CategoryGroupLists)
    print('P-Value for Anova is: ', AnovaResults[1])

#Linear Regression

##Simple Linear Model Attendence

In [ ]:
#use only participation since it has highest correlation
part_df=df[["First_Time_Attendees","Participated"]]
part_df

In [ ]:
#look at skewness of data
plt.figure(figsize=(20,6))

plt.subplot(131)
plt.hist(part_df["Participated"])
plt.title("Particiaption Distribution")

plt.subplot(132)
plt.hist(part_df["First_Time_Attendees"])
plt.title("First_Time_Attendees")

plt.subplot(133)
plt.scatter(part_df.Participated,part_df.First_Time_Attendees)
plt.ylabel("first time attendees")
plt.xlabel("participated")

In [ ]:
#take log to make data more normally distrbuted
part_df["First_Time_Attendees"] = np.log(part_df["First_Time_Attendees"])
part_df["Participated"] = np.log(part_df["Participated"])
part_df

In [ ]:
#treat inf values
#https://stackoverflow.com/questions/17477979/dropping-infinite-values-from-dataframes-in-pandas
part_df=part_df[part_df.First_Time_Attendees != -np.inf]
part_df=part_df[part_df.Participated != -np.inf]
part_df

In [ ]:
#plot new histogram to see changes
plt.figure(figsize=(20,6))

plt.subplot(1,3,1)
plt.hist(part_df["First_Time_Attendees"])
plt.title("First_Time_Attendees")

plt.subplot(1,3,2)
plt.hist(part_df["Participated"])
plt.title("Participated")

plt.subplot(1,3,3)
plt.scatter(part_df.Participated,part_df.First_Time_Attendees)
plt.ylabel("first time attendees")
plt.xlabel("participated")


In [ ]:
#define x and y and numpy array
slr_x=part_df[["Participated"]].values
slr_y=part_df[["First_Time_Attendees"]].values

In [ ]:
#divide into training and test set
slr_x_train, slr_x_test, slr_y_train, slr_y_test = train_test_split(slr_x,slr_y,test_size=.2, random_state=3)

In [ ]:
#model for training set
slr=linear_model.LinearRegression() #creating object for regression
slr.fit(slr_x_train,slr_y_train)  #true

In [ ]:
#Coefficient and Intercept
print("Coefficient is:", slr.coef_)
print("Intercept is:", slr.intercept_)

In [ ]:
#R-Square for training set
#https://www.datacamp.com/community/tutorials/essentials-linear-regression-python
slr.score(slr_x_train,slr_y_train)

In [ ]:
#create prediction
y_pred=slr.predict(slr_x_test)

In [ ]:
#plot to see regression line
plt.figure(figsize=(6,6))
plt.scatter(slr_x_train, slr_y_train)
plt.plot(slr_x_test,y_pred, color="red")
plt.xlabel("X Train")
plt.ylabel("Y Train")
plt.title("First Time Attendees Regression Line")

In [ ]:
#Model for test set
slr.fit(slr_x_test,slr_y_test)  #true

In [ ]:
#R-Square for test set
slr.score(slr_x_test,slr_y_test)  #true

In [ ]:
#compare y_test with predicted values
comp_df=pd.DataFrame({"Actual":slr_y_test.flatten(),"Predicted":y_pred.flatten()}).round(decimals = 3)
comp_df["Error"]=comp_df["Actual"]-comp_df["Predicted"]
comp_df.head()

In [ ]:
#R-Square for test set
print("FIRST TIME ATTENDEES~PARTICIPATION")
print("R Square:", slr.score(slr_x_test,slr_y_test).round(decimals=3))
#metrics
print('MAE:', metrics.mean_absolute_error(slr_y_test, y_pred).round(decimals=3))
print('MSE:', metrics.mean_squared_error(slr_y_test, y_pred).round(decimals=3))
print('RMSE:', np.sqrt(metrics.mean_squared_error(slr_y_test, y_pred)).round(decimals=3))

##Multiple Linear Model

In [ ]:
#pick variables
ml_df=df[["First_Time_Attendees","Participated","Major_Prospects"]]
ml_df

In [ ]:
#look at skewness of data
plt.figure(figsize=(20,6))

plt.subplot(131)
plt.hist(ml_df["Participated"])
plt.title("Particiaption Distribution")

plt.subplot(132)
plt.hist(ml_df["First_Time_Attendees"])
plt.title("First_Time_Attendees")

plt.subplot(133)
plt.hist(ml_df["Major_Prospects"])
plt.title("Major_Prospects")

In [ ]:
#take log of variables to make data more normal
ml_df["First_Time_Attendees"] = np.log(ml_df["First_Time_Attendees"])
ml_df["Participated"] = np.log(ml_df["Participated"])
ml_df["Major_Prospects"] = np.log(ml_df["Major_Prospects"])
ml_df

In [ ]:
#ml_df["First_Time_Attendees"].replace([np.inf, -np.inf], np.nan, inplace=True)
#ml_df = ml_df.replace(0, pd.np.nan).dropna(axis=0, how='any').fillna(0)

#ml_df["Participated"].replace([np.inf, -np.inf], np.nan, inplace=True)
#ml_df =  ml_df.replace(0, pd.np.nan).dropna(axis=0, how='any').fillna(0)

#ml_df["Major_Prospects"].replace([np.inf, -np.inf], np.nan, inplace=True)
#ml_df =  ml_df.replace(0, pd.np.nan).dropna(axis=0, how='any').fillna(0)
#ml_df

#Treat inf values
ml_df=ml_df[ml_df.First_Time_Attendees != -np.inf]
ml_df=ml_df[ml_df.Participated != -np.inf]
ml_df=ml_df[ml_df.Major_Prospects != -np.inf]
ml_df

In [ ]:
#dropping null values
ml_df=ml_df.dropna()
ml_df

In [ ]:
#plot to see changes in skewness
plt.figure(figsize=(20,6))

plt.subplot(1,3,1)
plt.hist(ml_df["First_Time_Attendees"])
plt.title("First_Time_Attendees")

plt.subplot(1,3,2)
plt.hist(ml_df["Participated"])
plt.title("Participated")

plt.subplot(1,3,3)
plt.hist(ml_df["Major_Prospects"])
plt.title("Major_Prospects")


In [ ]:
#define x and y variables and set them as numpy array
#convert to numpy array
ml_x=ml_df[["Participated","Major_Prospects"]].values
ml_y=ml_df[["First_Time_Attendees"]].values
print(ml_x.shape)
print(ml_y.shape)

In [ ]:
#divide into training and test set
ml_x_train, ml_x_test, ml_y_train, ml_y_test = train_test_split(ml_x,ml_y,test_size=.2, random_state=2)
print(ml_x_train.shape, ml_x_test.shape, ml_y_train.shape, ml_y_test.shape)

In [ ]:
#make regression instance
multreg= LinearRegression(fit_intercept=True)

In [ ]:
#train model
multreg.fit(ml_x_train,ml_y_train)

In [ ]:
#Coefficient and Intercept
print("Coefficient is:", multreg.coef_)
print("Intercept is:", multreg.intercept_)

In [ ]:
#R square ofr training model
multreg.score(ml_x_train,ml_y_train)

In [ ]:
#create prediction
y_mlpred=multreg.predict(ml_x_test)

In [ ]:
#test model
multreg.fit(ml_x_test,ml_y_test)

In [ ]:
#R Square for test model
multreg.score(ml_x_test,ml_y_test)

In [ ]:
#compare y_test with predicted values
mlcomp_df=pd.DataFrame({"Actual":ml_y_test.flatten(),"Predicted":y_mlpred.flatten()}).round(decimals = 3)
mlcomp_df["Error"]=mlcomp_df["Actual"]-mlcomp_df["Predicted"]
mlcomp_df.head(10)

In [ ]:
#R-Square for test set
print("FIRST TIME ATTENDEES~PARTICIPATION+MAJOR PROSPECTS")
print("R Square:", multreg.score(ml_x_test,ml_y_test).round(decimals=3))
#metrics
print('MAE:', metrics.mean_absolute_error(ml_y_test, y_mlpred).round(decimals=3))
print('MSE:', metrics.mean_squared_error(ml_y_test, y_mlpred).round(decimals=3))
print('RMSE:', np.sqrt(metrics.mean_squared_error(ml_y_test, y_mlpred)).round(decimals=3))

##Multiple Linear Model with Dummies

In [ ]:
#define new df with X and Y variables
cat=df[["First_Time_Attendees","Participated","Major_Prospects","Location_Code","Activity_Code"]]

In [ ]:
#take log of variables to make data more normal
cat["First_Time_Attendees"] = np.log(cat["First_Time_Attendees"])
cat["Participated"] = np.log(cat["Participated"])
cat["Major_Prospects"] = np.log(cat["Major_Prospects"])

In [ ]:
#Treat inf values
cat=cat[cat.First_Time_Attendees != -np.inf]
cat=cat[cat.Participated != -np.inf]
cat=cat[cat.Major_Prospects != -np.inf]
cat

In [ ]:
#dropping null values
cat=cat.dropna()
cat

In [ ]:
cat_X=cat[["Participated","Major_Prospects","Activity_Code","Location_Code"]]

In [ ]:
#get dummines
cat_X=pd.get_dummies(cat_X, columns=["Location_Code","Activity_Code"], drop_first=True)
cat_X

In [ ]:
#convert to numpy array
cat_X=cat_X.values
cat_Y=cat[["First_Time_Attendees"]].values

In [ ]:
print(cat_X.shape)
print(cat_Y.shape)

In [ ]:
#split data
cat_X_train, cat_X_test, cat_Y_train, cat_Y_test=train_test_split(cat_X,cat_Y,test_size=0.2, random_state=2)
print(cat_X_train.shape, cat_X_test.shape, cat_Y_train.shape, cat_Y_test.shape)

In [ ]:
finallr= LinearRegression()
finallr

In [ ]:
#train model
finallr.fit(cat_X_train,cat_Y_train)

In [ ]:
#Coefficient and Intercept
print("Coefficient is:", finallr.coef_)
print("Intercept is:", finallr.intercept_)

In [ ]:
#R square for training model
finallr.score(cat_X_train, cat_Y_train)

In [ ]:
#create prediction
y_finallrpred=finallr.predict(cat_X_test)

In [ ]:
#test model
finallr.fit(cat_X_test, cat_Y_test)

In [ ]:
#R Square for test model
finallr.score(cat_X_test, cat_Y_test)

In [ ]:
#compare y_test with predicted values
finalcomp_df=pd.DataFrame({"Actual":cat_Y_test.flatten(),"Predicted":y_finallrpred.flatten()}).round(decimals = 3)
finalcomp_df["Error"]=finalcomp_df["Actual"]-finalcomp_df["Predicted"]
finalcomp_df.head(10)

In [ ]:
#R-Square for test set
print("FIRST TIME ATTENDEES~PARTICIPATION+MAJOR PROSPECTS+LOCATION CODE+ACITIVITY CODE")
print("R Square:", finallr.score(cat_X_test, cat_Y_test).round(decimals=3))
#metrics
print('MAE:', metrics.mean_absolute_error(cat_Y_test, y_finallrpred).round(decimals=3))
print('MSE:', metrics.mean_squared_error(cat_Y_test, y_finallrpred).round(decimals=3))
print('RMSE:', np.sqrt(metrics.mean_squared_error(cat_Y_test, y_finallrpred)).round(decimals=3))

##First Time Attendees Linear Regression with dummies

In [ ]:
#X and Y arrays
X = df[[ 'Activity_Code', 'Location_Code', 'Participated', 'Major_Prospects']]
y = df['First_Time_Attendees']

In [ ]:
#https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd
OneHotEncoder(drop='first').fit(X).categories_

In [ ]:
#convert string to float 
X1 = X.apply(pd.to_numeric, errors='coerce')
Y1 = y.apply(pd.to_numeric, errors='coerce')
X1.fillna(0, inplace=True)
Y1.fillna(0, inplace=True)

In [ ]:
X1

In [ ]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.4, random_state=101)
#Create and train the model
lm = LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
# The coefficients
print('Coefficients: \n', lm.coef_)

**Predicting** **Test** **Data**

In [ ]:
 #Use lm.predict() to predict off the X_test set of the data
 predictions = lm.predict( X_test)

In [ ]:
 #Create a scatterplot of the real test values versus the predicted values. **
plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

**Evaluating** **the** **Model**

In [ ]:
#R-Square for test set
print("FIRST TIME ATTENDEES~PARTICIPATION + MAJOR PROSPECTS + LOCATION CODE + ACTIVITY CODE")
print("R Square:", lm.score(X_test,y_test).round(decimals=3))
#metrics
print('MAE:', metrics.mean_absolute_error(y_test, predictions).round(decimals=3))
print('MSE:', metrics.mean_squared_error(y_test, predictions).round(decimals=3))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)).round(decimals=3))

**Residuals**

In [ ]:
sns.distplot((y_test-predictions),bins=50);

**Conclusion**

In [ ]:
#Recreate the dataframe below
coeffecients = pd.DataFrame(lm.coef_,X.columns)
coeffecients.columns = ['Coeffecient']
coeffecients

##Simple Linear Model Major Prospects

In [ ]:
#use only participation since it has highest correlation
part_df=df[["Major_Prospects","Participated"]]
part_df

In [ ]:
#look at skewness of data
plt.figure(figsize=(20,6))

plt.subplot(131)
plt.hist(part_df["Participated"])
plt.title("Particiaption Distribution")

plt.subplot(132)
plt.hist(part_df["Major_Prospects"])
plt.title("Major_Prospects")

plt.subplot(133)
plt.scatter(part_df.Participated,part_df.Major_Prospects)
plt.ylabel("Major_Prospects")
plt.xlabel("participated")

In [ ]:
#take log to make data more normally distrbuted
part_df["Major_Prospects"] = np.log(part_df["Major_Prospects"])
part_df["Participated"] = np.log(part_df["Participated"])
part_df

In [ ]:
#treat inf values
part_df=part_df[part_df.Major_Prospects != -np.inf]
part_df=part_df[part_df.Participated != -np.inf]
part_df

In [ ]:
#plot new histogram to see changes
plt.figure(figsize=(20,6))

plt.subplot(1,3,1)
plt.hist(part_df["Major_Prospects"])
plt.title("Major_Prospects")

plt.subplot(1,3,2)
plt.hist(part_df["Participated"])
plt.title("Participated")

plt.subplot(1,3,3)
plt.scatter(part_df.Participated,part_df.Major_Prospects)
plt.ylabel("Major_Prospects")
plt.xlabel("participated")


In [ ]:
#define x and y and numpy array
slr_x=part_df[["Participated"]].values
slr_y=part_df[["Major_Prospects"]].values

In [ ]:
#divide into training and test set
slr_x_train, slr_x_test, slr_y_train, slr_y_test = train_test_split(slr_x,slr_y,test_size=.2, random_state=2)

In [ ]:
#model for training set
slr=linear_model.LinearRegression() #creating object for regression
slr.fit(slr_x_train,slr_y_train)  #true

In [ ]:
#Coefficient and Intercept
print("Coefficient is:", slr.coef_)
print("Intercept is:", slr.intercept_)

In [ ]:
#R-Square for training set
slr.score(slr_x_train,slr_y_train)

In [ ]:
#create prediction
y_pred=slr.predict(slr_x_test)

In [ ]:
#plot to see regression line
plt.figure(figsize=(6,6))
plt.scatter(slr_x_train, slr_y_train)
plt.plot(slr_x_test,y_pred, color="red")
plt.xlabel('X Train')
plt.ylabel('Y Train')
plt.title("Major_Prospects Regression Line")

In [ ]:
#Model for test set
slr.fit(slr_x_test,slr_y_test)  #true

In [ ]:
#R-Square for test set
slr.score(slr_x_test,slr_y_test)  #true

In [ ]:
#compare y_test with predicted values
comp_df=pd.DataFrame({"Actual":slr_y_test.flatten(),"Predicted":y_pred.flatten()}).round(decimals = 3)
comp_df["Error"]=comp_df["Actual"]-comp_df["Predicted"]
comp_df.head()

In [ ]:
#R-Square for test set
print("MAJOR PROSPECTS~PARTICIPATION")
print("R Square:", slr.score(slr_x_test,slr_y_test).round(decimals=3))
#metrics
print('MAE:', metrics.mean_absolute_error(slr_y_test, y_pred).round(decimals=3))
print('MSE:', metrics.mean_squared_error(slr_y_test, y_pred).round(decimals=3))
print('RMSE:', np.sqrt(metrics.mean_squared_error(slr_y_test, y_pred)).round(decimals=3))

##Multiple Linear Model-Major Prospects

In [ ]:
#pick variables
ml_df=df[["First_Time_Attendees","Participated","Major_Prospects"]]
ml_df

In [ ]:
#look at skewness of data
plt.figure(figsize=(20,6))

plt.subplot(131)
plt.hist(ml_df["Participated"])
plt.title("Particiaption Distribution")

plt.subplot(132)
plt.hist(ml_df["First_Time_Attendees"])
plt.title("First_Time_Attendees")

plt.subplot(133)
plt.hist(ml_df["Major_Prospects"])
plt.title("Major_Prospects")

In [ ]:
#take log of variables to make data more normal
ml_df["First_Time_Attendees"] = np.log(ml_df["First_Time_Attendees"])
ml_df["Participated"] = np.log(ml_df["Participated"])
ml_df["Major_Prospects"] = np.log(ml_df["Major_Prospects"])
ml_df

In [ ]:
#ml_df["First_Time_Attendees"].replace([np.inf, -np.inf], np.nan, inplace=True)
#ml_df = ml_df.replace(0, pd.np.nan).dropna(axis=0, how='any').fillna(0)

#ml_df["Participated"].replace([np.inf, -np.inf], np.nan, inplace=True)
#ml_df =  ml_df.replace(0, pd.np.nan).dropna(axis=0, how='any').fillna(0)

#ml_df["Major_Prospects"].replace([np.inf, -np.inf], np.nan, inplace=True)
#ml_df =  ml_df.replace(0, pd.np.nan).dropna(axis=0, how='any').fillna(0)
#ml_df

#Treat inf values
ml_df=ml_df[ml_df.First_Time_Attendees != -np.inf]
ml_df=ml_df[ml_df.Participated != -np.inf]
ml_df=ml_df[ml_df.Major_Prospects != -np.inf]
ml_df

In [ ]:

#dropping null values
ml_df=ml_df.dropna()
ml_df

In [ ]:
#plot to see changes in skewness
plt.figure(figsize=(20,6))

plt.subplot(1,3,1)
plt.hist(ml_df["First_Time_Attendees"])
plt.title("First_Time_Attendees")

plt.subplot(1,3,2)
plt.hist(ml_df["Participated"])
plt.title("Participated")

plt.subplot(1,3,3)
plt.hist(ml_df["Major_Prospects"])
plt.title("Major_Prospects")


In [ ]:
#define x and y variables and set them as numpy array
#convert to numpy array
ml_x=ml_df[["Participated","First_Time_Attendees"]].values
ml_y=ml_df[["Major_Prospects"]].values
print(ml_x.shape)
print(ml_y.shape)

In [ ]:
#divide into training and test set
ml_x_train, ml_x_test, ml_y_train, ml_y_test = train_test_split(ml_x,ml_y,test_size=.2, random_state=2)
print(ml_x_train.shape, ml_x_test.shape, ml_y_train.shape, ml_y_test.shape)

In [ ]:
#make regression instance
multreg= LinearRegression(fit_intercept=True)

In [ ]:
#train model
multreg.fit(ml_x_train,ml_y_train)

In [ ]:
#Coefficient and Intercept
print("Coefficient is:", multreg.coef_)
print("Intercept is:", multreg.intercept_)

In [ ]:
#R square ofr training model
multreg.score(ml_x_train,ml_y_train)

In [ ]:

#create prediction
y_mlpred=multreg.predict(ml_x_test)

In [ ]:
#test model
multreg.fit(ml_x_test,ml_y_test)

In [ ]:
#R Square for test model
multreg.score(ml_x_test,ml_y_test)

In [ ]:
#compare y_test with predicted values
mlcomp_df=pd.DataFrame({"Actual":ml_y_test.flatten(),"Predicted":y_mlpred.flatten()}).round(decimals = 3)
mlcomp_df["Error"]=mlcomp_df["Actual"]-mlcomp_df["Predicted"]
mlcomp_df.head(10)

In [ ]:
#R-Square for test set
print("FIRST TIME ATTENDEES~PARTICIPATION+MAJOR PROSPECTS")
print("R Square:", multreg.score(ml_x_test,ml_y_test).round(decimals=3))
#metrics
print('MAE:', metrics.mean_absolute_error(ml_y_test, y_mlpred).round(decimals=3))
print('MSE:', metrics.mean_squared_error(ml_y_test, y_mlpred).round(decimals=3))
print('RMSE:', np.sqrt(metrics.mean_squared_error(ml_y_test, y_mlpred)).round(decimals=3))

##Major Prospect Linear Regression with dummies

In [ ]:
#X and Y arrays
X = df[[ 'Activity_Code', 'Location_Code', 'Participated', 'First_Time_Attendees']]
y = df['Major_Prospects']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
OneHotEncoder(drop='first').fit(X).categories_

In [ ]:
#convert string to float 
X1 = X.apply(pd.to_numeric, errors='coerce')
Y1 = y.apply(pd.to_numeric, errors='coerce')
X1.fillna(0, inplace=True)
Y1.fillna(0, inplace=True)

In [ ]:
X1

In [ ]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.4, random_state=101)
#Create and train the model
lm = LinearRegression()
lm.fit(X_train,y_train)

**Residuals**

In [ ]:
sns.distplot((y_test-predictions),bins=50);

In [ ]:
#Recreate the dataframe below
coeffecients = pd.DataFrame(lm.coef_,X.columns)
coeffecients.columns = ['Coeffecient']
coeffecients